# 6 Deployment — Strategien & Architektur

## 6.1 Einleitung
Nachdem in den vorangegangenen Notebooks die Modellentwicklung und die Evaluation der Erklärmethoden (SHAP vs. Captum) im Fokus standen, widmet sich dieses Notebook der Überführung des Systems in eine produktive Umgebung. Da die Evaluation gezeigt hat, dass SHAP zwar qualitativ hochwertigere Ergebnisse liefert, aber rechenintensiv ist, und Captum schneller, aber weniger robust ist, muss die Deployment-Architektur diese Dualität berücksichtigen. Ziel ist ein theoretisches Konzept für einen skalierbaren, wartbaren und vertrauenswürdigen Sentiment-Service.

## 6.2 Anforderungen an das Deployment

### 6.2.1 Funktionale Anforderungen
- **Inferenz:** Bereitstellung von Sentiment-Vorhersagen (Positiv/Negativ) mit Konfidenzscores.
- **Erklärbarkeit:** On-Demand-Bereitstellung von Feature-Attributionen. Unterstützung verschiedener Modi (Schnell/Captum vs. Präzise/SHAP).
- **Reproduzierbarkeit:** Sicherstellung, dass jede Vorhersage und Erklärung auf eine spezifische Modell- und Datenversion zurückgeführt werden kann.

### 6.2.2 Nicht-funktionale Anforderungen
- **Latenz:** < 200ms für reine Inferenz, < 500ms für Captum-Erklärungen. SHAP-Erklärungen dürfen asynchron bereitgestellt werden.
- **Skalierbarkeit:** Horizontale Skalierung bei Lastspitzen (z.B. virale Tweets).
- **Wartbarkeit:** Modularer Aufbau zur unabhängigen Aktualisierung von Modell und Erklärkomponenten.

## 6.3 Architekturentwurf für XAI-Services
Um den unterschiedlichen Anforderungen an Geschwindigkeit und Genauigkeit gerecht zu werden, empfiehlt sich eine **Microservices-Architektur** mit getrennten Pfaden für Inferenz und Erklärung.

### 6.3.1 Service-Schnittstellen
1.  **Inference Service (Synchron):**
    *   REST/gRPC API.
    *   Input: Text.
    *   Output: Sentiment, Score, ID.
    *   Optimiert auf Durchsatz und geringe Latenz.
2.  **Explanation Service (Hybrid):**
    *   **Fast Path (Synchron):** Nutzt Captum (Layer Integrated Gradients) für sofortiges Feedback im UI.
    *   **Deep Dive Path (Asynchron):** Nutzt SHAP für detaillierte Audits oder Analysten-Dashboards. Anfragen werden in eine Queue (z.B. RabbitMQ/Kafka) gestellt und von Workern abgearbeitet.

### 6.3.2 Caching-Strategie
Da Erklärungen deterministisch für eine feste Modellversion sind, sollten Ergebnisse aggressiv gecached werden (z.B. Redis). Häufige Anfragen oder wiederkehrende Textmuster müssen nicht neu berechnet werden.

## 6.4 MLOps & Lifecycle Management
Der Betrieb von XAI-Modellen erfordert erweiterte Monitoring-Strategien.

### 6.4.1 Monitoring
- **Data Drift:** Überwachung der Eingabeverteilung (z.B. neue Slang-Begriffe).
- **Concept Drift:** Veränderung der Sentiment-Bedeutung über die Zeit.
- **Explanation Stability:** Überwachung der durchschnittlichen Attributions-Scores. Plötzliche Änderungen in den Heatmaps können auf Modellfehler hindeuten, noch bevor die Accuracy sinkt.

### 6.4.2 CI/CD Pipeline
- **Build:** Automatisierte Tests (Unit Tests, Integration Tests).
- **Evaluation Gate:** Automatischer Vergleich neuer Modellversionen gegen die Baseline aus Notebook 5 (Faithfulness, Robustheit).
- **Deployment:** Blue-Green Deployment, um neue Versionen risikofrei auszurollen.

## 6.5 Governance & Compliance
Der Einsatz von KI in Entscheidungsprozessen erfordert Transparenz.

- **Audit Trails:** Speicherung von (Input, Output, Explanation, ModelVersion) für jede kritische Entscheidung.
- **Human-in-the-Loop:** Bei niedriger Konfidenz oder widersprüchlichen Erklärungen (z.B. SHAP vs. Captum Divergenz) sollte der Fall zur manuellen Prüfung ausgesteuert werden.
- **Datenschutz:** Sicherstellung, dass in den Erklärungen keine personenbezogenen Daten (PII) ungewollt hervorgehoben oder gespeichert werden.

## 6.6 Roadmap zur Produktivsetzung

### Phase 1: MVP (Monat 1-2)
- Containerisierung des Modells (Docker).
- Bereitstellung des synchronen Inference-Services.
- Integration von Captum für einfache "Live"-Erklärungen.

### Phase 2: Advanced XAI (Monat 3-4)
- Implementierung der asynchronen SHAP-Worker.
- Aufbau des Caching-Layers.
- Integration in das Analysten-Dashboard.

### Phase 3: Industrialisierung (Monat 5+)
- Vollständiges Monitoring und Alerting.
- Automatisierte Retraining-Pipelines basierend auf Drift-Erkennung.
- Skalierung auf Kubernetes.

## 6.7 Fazit
Das Deployment von erklärbaren KI-Modellen erfordert eine sorgfältige Abwägung zwischen Nutzererfahrung (Latenz) und Erklärungsgüte (Faithfulness). Die in dieser Arbeit entwickelte hybride Architektur adressiert dieses Spannungsfeld durch die Kombination von schnellen Captum-Erklärungen für den Echtzeitbetrieb und präzisen SHAP-Analysen für Audits. Mit der skizzierten Roadmap und den Governance-Richtlinien ist der Grundstein für einen verantwortungsvollen produktiven Einsatz des Sentiment-Analyse-Systems gelegt.